# Introduction to MLops with zenML

## ML Pipelines with ZenML

In [3]:
!pip install --upgrade SQLAlchemy==1.4.41


In [4]:
%pip install "zenml[server]"
!zenml integration install sklearn -y
%pip install pyparsing==2.4.2 # requerid for Colab

import IPython

# Automatically restart kernel
IPython.Application.instance().kernel.do_shutdown( restart = True)

NumExpr defaulting to 2 threads.
⠙ Installing integrations...


{'status': 'ok', 'restart': True}

In [26]:
NGROK_TOKEN = "2e9SIxApc7XTxUFgdcC1rvwOWkj_qYg9AzEiYUCfrW1KKZQc"

In [28]:
from zenml.environment import Environment

if Environment.in_google_colab():

  !pip install pyngrok
  !ngrok authtoken {NGROK_TOKEN}

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


# ZenML Setup

In [7]:
!rm -rf .zen
!zenml init

NumExpr defaulting to 2 threads.
⠋ Initializing ZenML repository at /content.
⠸ Initializing ZenML repository at /content.
⠼ Initializing ZenML repository at /content.
⠴ Initializing ZenML repository at /content.
⠦ Initializing ZenML repository at /content.
⠧ Initializing ZenML repository at /content.
⠏ Initializing ZenML repository at /content.
⠋ Initializing ZenML repository at /content.
⠙ Initializing ZenML repository at /content.
Setting the repo active workspace to 'default'.
Setting the repo active stack to default.
ZenML repository initialized at /content.
⠹ Initializing ZenML repository at /content.
⠹ Initializing ZenML repository at /content.

The local active stack was initialized to 'default'. This local configuration will only take effect 
when you're running ZenML from the initialized repository root, or from a subdirectory. For more 
information on repositories and configurations, please visit 
https://docs.zenml.io/user-guide/starter-guide/understand-stacks.


## Experimentation ML

In [13]:
import numpy as np
from sklearn.base import ClassifierMixin
from sklearn.datasets import load_digits
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split

def train_test() -> None:
  """Train and test a scikit-learn SVC Classifier on Digits"""
  digits = load_digits()
  data = digits.images.reshape( len( digits.images ), -1)
  X_train, X_test, y_train, y_test = train_test_split( data, digits.target, test_size=0.2, shuffle=False )

  model = SVC( gamma=0.001 ) # gamma hyperparameter: smaller help to prevent overffiting
  model.fit( X_train, y_train)
  test_acc = model.score( X_test, y_test )
  print(f"Test accuracy: {test_acc}"  )

train_test()

Test accuracy: 0.9583333333333334


## Turning experiments into ML pipelines with ZenML

*Pipeline: Load -> Training -> Evaluate*

In [10]:
from zenml import step
from typing_extensions import Annotated
import pandas as pd
from typing import Tuple
import numpy as np

@step
def importer() -> Tuple[
    Annotated[np.ndarray, "X_train"],
    Annotated[np.ndarray, "X_test"],
    Annotated[np.ndarray, "y_train"],
    Annotated[np.ndarray, "y_test"],
]:
  """Load the digits dataset as numpy arrays"""
  digits = load_digits()
  data = digits.images.reshape( len( digits.images ), -1)
  X_train, X_test, y_train, y_test = train_test_split( data, digits.target, test_size=0.2, shuffle=False )
  return  X_train, X_test, y_train, y_test

@step
def svc_trainer(
    X_train: np.ndarray,
    y_train: np.ndarray
    ) -> ClassifierMixin:
  """Train a sklearn Classifier"""
  model = SVC( gamma=0.001 ) # gamma hyperparameter: smaller help to prevent overffiting
  model.fit( X_train, y_train)
  return  model

@step
def evaluator(
    X_test: np.ndarray,
    y_test: np.ndarray,
    model: ClassifierMixin,) -> float:
    """Calculate accuracy"""
    test_acc = model.score( X_test, y_test )
    print(f"Test accuracy: {test_acc}"  )

    return test_acc



In [16]:
from zenml import pipeline

@pipeline
def digits_pipeline():
  """Link all the steps together in a pipeline"""
  X_train, X_test, y_train, y_test = importer()
  model = svc_trainer(X_train=X_train, y_train = y_train)
  evaluator(X_test, y_test, model )


In [17]:
digists_svc_pipeline = digits_pipeline()


Initiating a new run for the pipeline: digits_pipeline.
Registered new version: (version 3).
Executing a new run.
Using user: default
Using stack: default
  orchestrator: default
  artifact_store: default
You can visualize your pipeline runs in the ZenML Dashboard. In order to try it locally, please run zenml up.
Using cached version of importer.
Step importer has started.
Step svc_trainer has started.
Step svc_trainer has finished in 0.726s.
Step evaluator has started.
Test accuracy: 0.9583333333333334
Step evaluator has finished in 0.668s.
Pipeline run has finished in 1.997s.


In [19]:
digists_svc_pipeline

PipelineRunResponseBody(created=datetime.datetime(2024, 3, 25, 0, 31, 6, 33337), updated=datetime.datetime(2024, 3, 25, 0, 31, 6, 33341), user=UserResponse(body=UserResponseBody(created=datetime.datetime(2024, 3, 25, 0, 18, 17, 500474), updated=datetime.datetime(2024, 3, 25, 0, 18, 17, 500480), active=True, activation_token=None, full_name='', email_opted_in=None, is_service_account=False, is_admin=True), metadata=None, resources=None, id=UUID('4028a627-aa25-40e5-b65e-d8dfd8cb8cb8'), permission_denied=False, name='default'), status=<ExecutionStatus.INITIALIZING: 'initializing'>, stack=StackResponse(body=StackResponseBody(created=datetime.datetime(2024, 3, 25, 0, 18, 16, 846547), updated=datetime.datetime(2024, 3, 25, 0, 18, 16, 846552), user=None), metadata=None, resources=None, id=UUID('0eb59b8b-5634-45b3-b551-a4bbc4e565e5'), permission_denied=False, name='default'), pipeline=PipelineResponse(body=PipelineResponseBody(created=datetime.datetime(2024, 3, 25, 0, 31, 5, 542054), updated=d

In [24]:
from zenml.environment import Environment

def start_zenml_dashboard( port=8237):
  if Environment.in_google_colab():
    from pyngrok import ngrok

    public_url = ngrok.connect(port)
    print(f"\x1b[31mIm Colab, use this URL instead: {public_url}!\x1b[0m")
    !zenml up --blocking --port {port}

  else:
    !zenml up --port {--port}

In [29]:
start_zenml_dashboard()

INFO:pyngrok.ngrok:Opening tunnel named: http-8237-b3a65c98-681a-41fe-9c0f-74d195839b57


Opening tunnel named: http-8237-b3a65c98-681a-41fe-9c0f-74d195839b57


INFO:pyngrok.process.ngrok:t=2024-03-25T00:45:06+0000 lvl=info msg="no configuration paths supplied"


t=2024-03-25T00:45:06+0000 lvl=info msg="no configuration paths supplied"


INFO:pyngrok.process.ngrok:t=2024-03-25T00:45:06+0000 lvl=info msg="using configuration at default config path" path=/root/.config/ngrok/ngrok.yml


...

t=2024-03-25T00:54:01+0000 lvl=info msg="join connections" obj=join id=a81e853673be l=127.0.0.1:8237 r=[2800:300:66d7:bd0::1]:56110
INFO:     2800:300:66d7:bd0::1:0 - "GET /media/arrowOpen.6ceef0af.svg HTTP/1.1" 200 OK


INFO:pyngrok.process.ngrok:t=2024-03-25T00:54:59+0000 lvl=info msg="received stop request" obj=app stopReq="{err:<nil> restart:false}"


t=2024-03-25T00:54:59+0000 lvl=info msg="received stop request" obj=app stopReq="{err:<nil> restart:false}"


INFO:pyngrok.process.ngrok:t=2024-03-25T00:54:59+0000 lvl=info msg="session closing" obj=tunnels.session err=nil


t=2024-03-25T00:54:59+0000 lvl=info msg="session closing" obj=tunnels.session err=nil
INFO:     Shutting down
INFO:     Finished server process [7943]
ERROR:    Traceback (most recent call last):
  File "/usr/lib/python3.10/asyncio/runners.py", line 44, in run
    return loop.run_until_complete(main)
  File "uvloop/loop.pyx", line 1511, in uvloop.loop.Loop.run_until_complete
  File "uvloop/loop.pyx", line 1504, in uvloop.loop.Loop.run_until_complete
  File "uvloop/loop.pyx", line 1377, in uvloop.loop.Loop.run_forever
  File "uvloop/loop.pyx", line 555, in uvloop.loop.Loop._run
  File "uvloop/loop.pyx", line 474, in uvloop.loop.Loop._on_idle
  File "uvloop/cbhandles.pyx", line 83, in uvloop.loop.Handle._run
  File "uvloop/cbhandles.pyx", line 63, in uvloop.loop.Handle._run
  File "/usr/local/lib/python3.10/dist-packages/uvicorn/server.py", line 68, in serve
    with self.capture_signals():
  File "/usr/lib/python3.10/contextlib.py", line 142, in __exit__
    next(self.gen)
  File "/usr/